# Resize image and add bg

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

_img = 'images/xbox.jpg'
image = Image.open(_img)
image.thumbnail((224, 224), Image.ANTIALIAS)
background = Image.new('RGB', (224, 224), (255, 255, 255))
background.paste(
    image, (int((224 - image.size[0]) / 2), int((224 - image.size[1]) / 2))
)
background.save("images/xbox_re", "JPEG")
imgplot = plt.imshow(background)

In [ ]:
imgplot = plt.imshow(background, interpolation="bicubic")

# Image augmentation using keras

In [ ]:
PATH = '/home/shiyue/Desktop/workplace/vgg16_v1/images/'
dir_list = glob.glob(PATH+'*')
dir_list

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import img_to_array, load_img
import glob

PATH = '/home/shiyue/Desktop/workplace/vgg16_v1/images/'
dir_list = glob.glob(PATH+'*')
# SAVE_PATH = '/home/shiyue/Desktop/workplace/vgg16_v1/images/IA/'
# name_list = glob.glob(PATH+'*/*')
# 设置生成器参数
datagen = image.ImageDataGenerator(
                            rescale=1./255,
                            rotation_range=90 ,
                            fill_mode='nearest',
                            shear_range=0.2,
                            zoom_range=0.2,
                            width_shift_range=0.2,
                            height_shift_range=0.2)

for dir_name in dir_list:
    brand_list = glob.glob(dir_name+'/*')
    for brand_name in brand_list:
        device_list = glob.glob(brand_name+'/*')
        for device_name in device_list:
            img_list = glob.glob(device_name+'/*')
            for img_name in img_list:
            # TODO, import num_array as img
                img = load_img(img_name)
                img = img_to_array(img)
                img = img.reshape((1,)+ img.shape)
                gen_data = datagen.flow(x=img,
                                        batch_size = 1,
                                        shuffle = True,
                                        save_to_dir = device_name,
                                        save_format = 'jpeg'
                                        )
                count = 0
                for batch in gen_data:
                    count+=1
                    if(count>5):
                        print('save')
                        break;


save
save
save
save
save
save
save
save
save
save
save
save


# Test with pre-trained model

In [ ]:
import tensorflow as tf

model = tf.keras.applications.VGG16()
imgfile = 'images/ps3.jpg'
image = tf.keras.preprocessing.image.load_img(
    imgfile, target_size=(224, 224))
image

In [ ]:
import numpy as np

array = tf.keras.preprocessing.image.img_to_array(image)
array = np.expand_dims(array, axis=0)
# preprocess_input should be used 
array = tf.keras.applications.vgg16.preprocess_input(array)
probabilities = model.predict(array)
tf.keras.applications.vgg16.decode_predictions(probabilities)

# Save & restore model
## Freeze the model and change the final layer only

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model

model = VGG16(
    include_top=True,
    weights='imagenet'
)

vgg_model = Model(inputs=model.input, outputs=model.get_layer('fc2').output)
for layer in vgg_model.layers:
    layer.trainable = False

#Create your own input format (here 3x224x224)
class_num = 10
vgginput = Input(shape=(224,224,3),name = 'image_input')
#Use the generated model 
output_vgg16_conv = vgg_model(vgginput)
#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(class_num, activation='softmax', name='predictions')(x)
#Create your own model 
my_model = Model(inputs=vgginput, outputs=x)
my_model.summary()

/home/shiyue/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
model_1 (Model)              (None, 4096)              134260544 
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
predictions (Dense)          (None, 10)                40970     
Total params: 134,301,514
Trainable params: 40,970
Non-trainable params: 134,260,544
_________________________________________________________________


In [ ]:
class_num = 10
for layer in model.layers:
#     print(layer.get_config())
    if(layer.name!='predictions'):
#         print(layer.name)
        layer.trainable = False
    else:
        config = layer.get_config()
        config['units'] = 10
        layer. = Dense.from_config(config)
    
model.summary()

## Freeze the conv level

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model

model = VGG16(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None
#     ,classes=1000
)
model.summary()
for layer in model_vgg16_conv.layers:
    layer.trainable = False

In [ ]:
# https://github.com/keras-team/keras/issues/4465
import tensorflow as tf
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
class_num = 10
model_vgg16_conv = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
model_vgg16_conv.summary()

#Create your own input format (here 3x224x224)
vgginput = Input(shape=(224,224,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(vgginput)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(class_num, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(inputs=vgginput, outputs=x)
# my_model = Model(vgginput, x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()


#Then training with your data ! 

## simple save and restore
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [ ]:
import os.path 

result_path = './results'
if not os.path.isdir(result_path):
    os.makedirs(result_path)

# serialize model to JSON
model_json = my_model.to_json()
with open(os.path.join(result_path, "vgg_self_model.json"), "w") as json_file:
    json_file.write(model_json)
my_model.save_weights(os.path.join(result_path, "vgg16_self.h5"))
print("Saved model to disk")

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.keras.models import model_from_json

# load json and create model
json_file = open(os.path.join(result_path, 'vgg_self_model.json'), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(os.path.join(result_path, "vgg16_self.h5"))
print("Loaded model from disk")

loaded_model.summary()

# Freeze conv and check with tensorboard

In [ ]:
# https://github.com/keras-team/keras/issues/4465
import tensorflow as tf
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
class_num = 10
model_vgg16_conv = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
for layer in model_vgg16_conv.layers:
    layer.trainable = False
model_vgg16_conv.summary()
#Create your own input format (here 3x224x224)
vgginput = Input(shape=(224,224,3),name = 'image_input')
#Use the generated model 
output_vgg16_conv = model_vgg16_conv(vgginput)

with tf.variable_scope("New_FCs"):
    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(class_num, activation='softmax', name='predictions')(x)
updatable_variables = tf.get_collection(
                        tf.GraphKeys.GLOBAL_VARIABLES,
                        scope='New_FCs')
#Create your own model 
my_model = Model(inputs=vgginput, outputs=x)
# my_model = Model(vgginput, x)
#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

